In [1]:
import sys
import os
from git_root import git_root

import numpy as np
import pandas as pd
import tensorflow as tf

sys.path.append(git_root("utils"))
from utils import load_params, load_config

sys.path.append(git_root("training"))
from fetch_data import fetch_data_local, fetch_format_stacked
from fetch_data import to_numpy_arrays, prepare_tf_dataset

import json

print(tf.__version__)

2.0.0


# Generating the predictions for the test sets

In [2]:
params = load_params()

angles = params["GLCM"]["mel_map"]["angles_in_deg"]

place_holder = {
    "spectrogram": {0: []},
    "mel_map": {
        angle: [] for angle in angles
    }
}

In [3]:
from tensorflow.keras.models import load_model
sys.path.append(git_root("models"))
from affine_scalar_layer import AffineScalar


mapping_for_saved_models = {'spectrogram': 'spec',
                           'mel_map': 'mel'}

for map_type in place_holder:
    for angle in place_holder[map_type]:
        #Fetch the data
        df = fetch_data_local(
            map_type=map_type, train=False, angle=angle
        )
        #Fetch the trained model
        model_path = git_root('models','saved_models',
                                  'glcm_{}_angle{}.h5'.format(mapping_for_saved_models[map_type],angle))
        net = load_model(model_path,  custom_objects={'AffineScalar':AffineScalar()})
        #For some reason, I need to cast my tensor to float32 for predict (and not for fit ?!)
        dataset = tf.data.Dataset.from_tensors(tf.dtypes.cast(to_numpy_arrays(df)[0], tf.float32))
        #Generate predictions
        y = net.predict(dataset)
        preds = pd.DataFrame(y)
        preds.columns = ["class{}".format(i) for i in range(10)]
        
        #Concatenate the prediction to our dataset
        df = df.merge(preds, left_index=True, right_index=True)
        df = df.drop(['maps'], axis=1)
        
        save_path=git_root("evaluation","evaluation_results","results_{}_{}.csv".format(map_type,angle))
        print("Saving to: {}".format(save_path))
        df.to_csv(save_path)

Fetching: data_spectrogram_angle_0_test.json
Saving to: /Users/arnaudstiegler/Desktop/deep-music-classification/evaluation/evaluation_results/results_spectrogram_0.csv
Fetching: data_mel_map_angle_0_test.json
Saving to: /Users/arnaudstiegler/Desktop/deep-music-classification/evaluation/evaluation_results/results_mel_map_0.csv
Fetching: data_mel_map_angle_45_test.json
Saving to: /Users/arnaudstiegler/Desktop/deep-music-classification/evaluation/evaluation_results/results_mel_map_45.csv
Fetching: data_mel_map_angle_90_test.json
Saving to: /Users/arnaudstiegler/Desktop/deep-music-classification/evaluation/evaluation_results/results_mel_map_90.csv
Fetching: data_mel_map_angle_135_test.json
Saving to: /Users/arnaudstiegler/Desktop/deep-music-classification/evaluation/evaluation_results/results_mel_map_135.csv


In [4]:
#Temporary list to stack the mel-maps
temp = []

for i in range(len(angles)):
    temp.append(fetch_data_local(
            map_type='mel_map', train=False, angle=angles[i]
        )['maps'])

stacked_mel_maps = (
    [
        np.stack([
            np.squeeze(temp[j][i]) for j in range(len(angles))
        ], -1) for i in range(1400)
    ]
)

Fetching: data_mel_map_angle_0_test.json
Fetching: data_mel_map_angle_45_test.json
Fetching: data_mel_map_angle_90_test.json
Fetching: data_mel_map_angle_135_test.json


In [5]:
#For stacked model

#Fetch the trained model
model_path = git_root('models','saved_models',
                                  'glcm_{}_stacked.h5'.format(mapping_for_saved_models['mel_map']))
net = load_model(model_path,  custom_objects={'AffineScalar':AffineScalar()})
#For some reason, I need to cast my tensor to float32 for predict (and not for fit ?!)
#dataset = tf.data.Dataset.from_tensors(tf.dtypes.cast(fetch_format_stacked(train=False), tf.float32))
dataset = tf.data.Dataset.from_tensor_slices(tf.dtypes.cast(stacked_mel_maps, tf.float32)).batch(32)
#Generate predictions
y = net.predict(dataset)
preds = pd.DataFrame(y)
preds.columns = ["class{}".format(i) for i in range(10)]
        
#Concatenate the prediction to our dataset
df = fetch_data_local(map_type='mel_map', train=False, angle='0').loc[:,['filename','split_id','genre']].merge(preds,left_index=True, right_index=True)
        
save_path=git_root("evaluation","evaluation_results","results_{}_{}.csv".format('mel_map','stacked'))
print("Saving to: {}".format(save_path))
df.to_csv(save_path)

Fetching: data_mel_map_angle_0_test.json
Saving to: /Users/arnaudstiegler/Desktop/deep-music-classification/evaluation/evaluation_results/results_mel_map_stacked.csv


# Results for Reference Models

In [6]:
config = load_config()

label_names = config["genres"]
label_to_idx = dict((name, index) for index, name in enumerate(label_names))
idx_to_label = {v: k for k, v in label_to_idx.items()}

class_columns = ["class{}".format(i) for i in range(10)]

In [7]:
idx_to_label

{0: 'blues',
 1: 'classical',
 2: 'country',
 3: 'disco',
 4: 'hiphop',
 5: 'jazz',
 6: 'metal',
 7: 'pop',
 8: 'reggae',
 9: 'rock'}

## Piece-level Results

In [8]:
from sklearn.metrics import classification_report

piece_level_acc = {"spectrogram": {0: {}},
    "mel_map": {
        angle: {} for angle in angles
    }}

piece_level_acc['mel_map']['stacked'] = {}

for map_type in piece_level_acc:
    for angle in piece_level_acc[map_type]:
        filepath=git_root("evaluation","evaluation_results","results_{}_{}.csv".format(map_type,angle))
        df = pd.read_csv(filepath, index_col=0)
        
        #Generate the label by taking highest prob
        df['pred'] = np.argmax(df[class_columns].values, axis=1)
        
        df['label'] = df['genre'].apply(lambda x: label_to_idx[x])
        
        report = classification_report(df['label'], df['pred'], output_dict=True)
        piece_level_acc[map_type][angle]['accuracy'] = report['accuracy']
        
        piece_level_acc[map_type][angle]['class_precision'] = {}
        piece_level_acc[map_type][angle]['class_recall'] = {}
        
        for i in range(10):
            piece_level_acc[map_type][angle]['class_precision'][idx_to_label[i]] = report[str(i)]['precision']
            piece_level_acc[map_type][angle]['class_recall'][idx_to_label[i]] = report[str(i)]['recall']

### Piece-level accuracy for Mel-maps

In [9]:
pd.DataFrame(piece_level_acc['mel_map']).T['accuracy']

0          0.437857
45         0.402143
90         0.383571
135        0.402857
stacked    0.439286
Name: accuracy, dtype: object

### Piece-level accuracy for spectrogram

In [10]:
pd.DataFrame(piece_level_acc['spectrogram']).T['accuracy']

0    0.464286
Name: accuracy, dtype: object

### Piece-level precision per genre for mel-maps

In [11]:
x = []
for angle in piece_level_acc[map_type]:
    x.append(piece_level_acc['mel_map'][angle]['class_precision'])
    
piece_level_mel_maps = pd.DataFrame(x)
piece_level_mel_maps['angle'] = pd.Series(['0','45','90','135','stacked'])
piece_level_mel_maps.set_index('angle', inplace=True)
piece_level_mel_maps

,blues,classical,country,disco,hiphop,jazz,metal,pop,reggae,rock
angle,,,,,,,,,,
0,0.619048,0.729730,0.369748,0.346154,0.281250,0.402778,0.729730,0.343849,0.383420,0.183206
45,0.490566,0.614865,0.257143,0.234043,0.250000,0.394366,0.759259,0.377850,0.355828,0.156716
90,0.514706,0.669492,0.280000,0.164384,0.217054,0.370588,0.707143,0.372611,0.364238,0.172840
135,0.521739,0.647436,0.339130,0.260870,0.145455,0.398810,0.761589,0.310897,0.333333,0.224138
stacked,0.441860,0.685535,0.369128,0.473684,0.425287,0.432203,0.707792,0.358974,0.366667,0.112245


### Piece-level recall per genre for mel-maps

In [12]:
x = []
for angle in piece_level_acc[map_type]:
    x.append(piece_level_acc['mel_map'][angle]['class_recall'])
    
piece_level_mel_maps = pd.DataFrame(x)
piece_level_mel_maps['angle'] = pd.Series(['0','45','90','135','stacked'])
piece_level_mel_maps.set_index('angle', inplace=True)
piece_level_mel_maps

,blues,classical,country,disco,hiphop,jazz,metal,pop,reggae,rock
angle,,,,,,,,,,
0,0.371429,0.771429,0.314286,0.128571,0.128571,0.414286,0.771429,0.778571,0.528571,0.171429
45,0.185714,0.650000,0.192857,0.157143,0.164286,0.400000,0.878571,0.828571,0.414286,0.150000
90,0.250000,0.564286,0.150000,0.085714,0.200000,0.450000,0.707143,0.835714,0.392857,0.200000
135,0.171429,0.721429,0.278571,0.171429,0.057143,0.478571,0.821429,0.692857,0.450000,0.185714
stacked,0.271429,0.778571,0.392857,0.192857,0.264286,0.364286,0.778571,0.800000,0.471429,0.078571


### Piece-level precision per genre for spectrograms

In [13]:
pd.Series(piece_level_acc['spectrogram'][0]['class_precision'])

blues        0.500000
classical    0.843537
country      0.458333
disco        0.339450
hiphop       0.263158
jazz         0.460674
metal        0.793548
pop          0.368078
reggae       0.413265
rock         0.120000
dtype: float64

### Piece-level recall per genre for spectrograms

In [14]:
pd.Series(piece_level_acc['spectrogram'][0]['class_recall'])

blues        0.192857
classical    0.885714
country      0.235714
disco        0.264286
hiphop       0.107143
jazz         0.585714
metal        0.878571
pop          0.807143
reggae       0.578571
rock         0.107143
dtype: float64

In [15]:
#Dump the results
with open(git_root('evaluation','evaluation_results','glcm_piece_level_results.json'), 'w') as fp:
    json.dump(piece_level_acc, fp, indent=4)

## Track-level Results

In [16]:
#Generate a table that contains mapping (track: genre)
track_mapping = df[['filename','genre']].drop_duplicates().reset_index(drop=True)

In [17]:
track_level_acc = {"spectrogram": {0: {}},
    "mel_map": {
        angle: {} for angle in angles
    }}

track_level_acc['mel_map']['stacked'] = {}

for map_type in track_level_acc:
    for angle in track_level_acc[map_type]:
        filepath=git_root("evaluation","evaluation_results","results_{}_{}.csv".format(map_type,angle))
        df = pd.read_csv(filepath, index_col=0)
        #Group by track to get track_level probs
        #Note that we SUM the probs (rather than AVG them)
        df_per_track = df.groupby('filename').sum()
        #Label is the highest prob
        df_per_track['pred'] = np.argmax(df_per_track[class_columns].values, axis=1)
        #Merge back to get the track_names
        df_result = df_per_track.merge(track_mapping, on='filename')
        df_result['label'] = df_result['genre'].apply(lambda x: label_to_idx[x])
        
        report = classification_report(df_result['label'], df_result['pred'], output_dict=True)
        track_level_acc[map_type][angle]['accuracy'] = report['accuracy']
        
        track_level_acc[map_type][angle]['class_precision'] = {}
        track_level_acc[map_type][angle]['class_recall'] = {}
        
        for i in range(10):
            track_level_acc[map_type][angle]['class_precision'][idx_to_label[i]] = report[str(i)]['precision']
            track_level_acc[map_type][angle]['class_recall'][idx_to_label[i]] = report[str(i)]['recall']

### Track-level accuracy for Mel-maps

In [18]:
pd.DataFrame(track_level_acc['mel_map']).T['accuracy']

0          0.52
45         0.42
90         0.41
135        0.46
stacked     0.5
Name: accuracy, dtype: object

### Track-level accuracy for spectrogram

In [19]:
pd.DataFrame(track_level_acc['spectrogram']).T['accuracy']

0    0.5
Name: accuracy, dtype: object

### Track-level precision per genre for mel-maps

In [20]:
x = []
for angle in track_level_acc[map_type]:
    x.append(track_level_acc['mel_map'][angle]['class_precision'])
    
track_level_mel_maps = pd.DataFrame(x)
track_level_mel_maps['angle'] = pd.Series(['0','45','90','135','stacked'])
track_level_mel_maps.set_index('angle', inplace=True)
track_level_mel_maps

,blues,classical,country,disco,hiphop,jazz,metal,pop,reggae,rock
angle,,,,,,,,,,
0,0.833333,0.818182,0.500000,0.500000,0.333333,0.545455,0.769231,0.307692,0.428571,0.250000
45,0.500000,0.615385,0.200000,0.200000,0.200000,0.500000,0.769231,0.360000,0.416667,0.100000
90,0.333333,0.666667,0.000000,0.166667,0.272727,0.500000,0.692308,0.375000,0.375000,0.111111
135,0.500000,0.692308,0.500000,0.285714,0.000000,0.500000,0.769231,0.347826,0.333333,0.200000
stacked,0.600000,0.600000,0.444444,0.666667,0.600000,0.500000,0.769231,0.320000,0.500000,0.200000


### Track-level recall per genre for mel-maps

In [21]:
x = []
for angle in track_level_acc[map_type]:
    x.append(track_level_acc['mel_map'][angle]['class_recall'])
    
track_level_mel_maps = pd.DataFrame(x)
track_level_mel_maps['angle'] = pd.Series(['0','45','90','135','stacked'])
track_level_mel_maps.set_index('angle', inplace=True)
track_level_mel_maps

,blues,classical,country,disco,hiphop,jazz,metal,pop,reggae,rock
angle,,,,,,,,,,
0,0.5,0.9,0.4,0.2,0.1,0.6,1.0,0.8,0.6,0.1
45,0.1,0.8,0.1,0.1,0.1,0.5,1.0,0.9,0.5,0.1
90,0.1,0.8,0.0,0.1,0.3,0.6,0.9,0.9,0.3,0.1
135,0.2,0.9,0.3,0.2,0.0,0.6,1.0,0.8,0.5,0.1
stacked,0.3,0.9,0.4,0.2,0.3,0.4,1.0,0.8,0.6,0.1


### Track-level accuracy per genre for spectrograms

In [22]:
pd.Series(track_level_acc['spectrogram'][0]['class_precision'])

blues        0.500000
classical    0.900000
country      1.000000
disco        0.428571
hiphop       0.500000
jazz         0.466667
metal        0.818182
pop          0.347826
reggae       0.466667
rock         0.090909
dtype: float64

### Track-level recall per genre for spectrograms

In [23]:
pd.Series(track_level_acc['spectrogram'][0]['class_recall'])

blues        0.1
classical    0.9
country      0.4
disco        0.3
hiphop       0.1
jazz         0.7
metal        0.9
pop          0.8
reggae       0.7
rock         0.1
dtype: float64

In [24]:
#Dump the results
with open(git_root('evaluation','evaluation_results','glcm_track_level_results.json'), 'w') as fp:
    json.dump(track_level_acc, fp, indent=4)

# Results for our own models

## Model with stacked input

In [25]:
model = 'stacked_final'

#Fetch the trained model
model_path = git_root('models','saved_models',
                                  'stacked_final.h5')
net = load_model(model_path)
#For some reason, I need to cast my tensor to float32 for predict (and not for fit ?!)
#dataset = tf.data.Dataset.from_tensors(tf.dtypes.cast(fetch_format_stacked(train=False), tf.float32))
dataset = tf.data.Dataset.from_tensor_slices(tf.dtypes.cast(stacked_mel_maps, tf.float32)).batch(32)
#Generate predictions
y = net.predict(dataset)
preds = pd.DataFrame(y)
preds.columns = ["class{}".format(i) for i in range(10)]
        
#Concatenate the prediction to our dataset
df = fetch_data_local(map_type='mel_map', train=False, angle='0').loc[:,['filename','split_id','genre']].merge(preds,left_index=True, right_index=True)
        
save_path=git_root("evaluation","evaluation_results","results_{}.csv".format(model))
print("Saving to: {}".format(save_path))
df.to_csv(save_path)

Fetching: data_mel_map_angle_0_test.json
Saving to: /Users/arnaudstiegler/Desktop/deep-music-classification/evaluation/evaluation_results/results_stacked_final.csv


## Model with concatenated architecture

In [32]:
model = 'concatenated_final'

#Fetch the trained model
model_path = git_root('models','saved_models',
                                  'concatenated_final.h5')
net = load_model(model_path, custom_objects={'AffineScalar':AffineScalar()})
#For some reason, I need to cast my tensor to float32 for predict (and not for fit ?!)
#dataset = tf.data.Dataset.from_tensors(tf.dtypes.cast(fetch_format_stacked(train=False), tf.float32))

x0 = np.stack(to_numpy_arrays(fetch_data_local(map_type='mel_map', train=False, angle='0'))[0]).astype(float)
x1 = np.stack(to_numpy_arrays(fetch_data_local(map_type='mel_map', train=False, angle='45'))[0]).astype(float)
x2 = np.stack(to_numpy_arrays(fetch_data_local(map_type='mel_map', train=False, angle='90'))[0]).astype(float)
x3 = np.stack(to_numpy_arrays(fetch_data_local(map_type='mel_map', train=False, angle='135'))[0]).astype(float)

Fetching: data_mel_map_angle_0_test.json
Fetching: data_mel_map_angle_45_test.json
Fetching: data_mel_map_angle_90_test.json
Fetching: data_mel_map_angle_135_test.json


In [33]:
#Generate predictions
y = net.predict([x0,x1,x2,x3])
preds = pd.DataFrame(y)
preds.columns = ["class{}".format(i) for i in range(10)]
        
#Concatenate the prediction to our dataset
df = fetch_data_local(map_type='mel_map', train=False, angle='0').loc[:,['filename','split_id','genre']].merge(preds,left_index=True, right_index=True)
        
save_path=git_root("evaluation","evaluation_results","results_{}.csv".format(model))
print("Saving to: {}".format(save_path))
df.to_csv(save_path)

Fetching: data_mel_map_angle_0_test.json
Saving to: /Users/arnaudstiegler/Desktop/deep-music-classification/evaluation/evaluation_results/results_concatenated_final.csv


## Piece-level accuracy

In [36]:
piece_new_models_acc={'stacked_final':{},
               'concatenated_final':{}}

for model in piece_new_models_acc:
    filepath=git_root("evaluation","evaluation_results","results_{}.csv".format(model))
    df = pd.read_csv(filepath, index_col=0)

    df['pred'] = np.argmax(df[class_columns].values, axis=1)

    df['label'] = df['genre'].apply(lambda x: label_to_idx[x])

    report = classification_report(df['label'], df['pred'], output_dict=True)
    piece_new_models_acc[model]['accuracy'] = report['accuracy']

    piece_new_models_acc[model]['class_precision'] = {}
    piece_new_models_acc[model]['class_recall'] = {}

    for i in range(10):
        piece_new_models_acc[model]['class_precision'][idx_to_label[i]] = report[str(i)]['precision']
        piece_new_models_acc[model]['class_recall'][idx_to_label[i]] = report[str(i)]['recall']


### Piece-level accuracy for our models

In [37]:
pd.DataFrame(piece_new_models_acc).T['accuracy']

stacked_final         0.362857
concatenated_final    0.509286
Name: accuracy, dtype: object

### Piece-level precision per genre for our models

In [39]:
x = []
for model in piece_new_models_acc:
    x.append(piece_new_models_acc[model]['class_precision'])
    
piece_level_models = pd.DataFrame(x)
piece_level_models['model'] = pd.Series(['stacked input', 'concatenated architecture'])
piece_level_models.set_index('model', inplace=True)
piece_level_models

,blues,classical,country,disco,hiphop,jazz,metal,pop,reggae,rock
model,,,,,,,,,,
stacked input,0.537313,0.828283,0.322344,0.137652,0.191176,0.510791,1.000000,0.465753,0.340426,0.211765
concatenated architecture,0.508621,0.900763,0.521008,0.290909,0.235294,0.549669,0.791946,0.441606,0.447059,0.270677


### Piece-level recall per genre for our models

In [40]:
x = []
for model in piece_new_models_acc:
    x.append(piece_new_models_acc[model]['class_recall'])
    
piece_level_models = pd.DataFrame(x)
piece_level_models['model'] = pd.Series(['stacked input', 'stacked architecture'])
piece_level_models.set_index('model', inplace=True)
piece_level_models

,blues,classical,country,disco,hiphop,jazz,metal,pop,reggae,rock
model,,,,,,,,,,
stacked input,0.257143,0.585714,0.628571,0.242857,0.092857,0.507143,0.142857,0.242857,0.800000,0.128571
stacked architecture,0.421429,0.842857,0.442857,0.114286,0.171429,0.592857,0.842857,0.864286,0.542857,0.257143


In [41]:
with open(git_root('evaluation','evaluation_results','piece_new_models_results.json'), 'w') as fp:
    json.dump(piece_new_models_acc, fp, indent=4)

## Track-level accuracy

In [43]:
track_new_models_acc={'stacked_final':{},
               'concatenated_final':{}}

for model in track_new_models_acc:
    filepath=git_root("evaluation","evaluation_results","results_{}.csv".format(model))
    df = pd.read_csv(filepath, index_col=0)

    df_per_track = df.groupby('filename').sum()
    #Label is the highest prob
    df_per_track['pred'] = np.argmax(df_per_track[class_columns].values, axis=1)
    #Merge back to get the track_names
    df_result = df_per_track.merge(track_mapping, on='filename')
    df_result['label'] = df_result['genre'].apply(lambda x: label_to_idx[x])


    report = classification_report(df_result['label'], df_result['pred'], output_dict=True)
    track_new_models_acc[model]['accuracy'] = report['accuracy']
        
    track_new_models_acc[model]['class_precision'] = {}
    track_new_models_acc[model]['class_recall'] = {}
        
    for i in range(10):
        track_new_models_acc[model]['class_precision'][idx_to_label[i]] = report[str(i)]['precision']
        track_new_models_acc[model]['class_recall'][idx_to_label[i]] = report[str(i)]['recall']


/Users/arnaudstiegler/miniconda3/envs/music_classification/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Track-level accuracy for our models

In [44]:
pd.DataFrame(track_new_models_acc).T['accuracy']

stacked_final         0.36
concatenated_final    0.57
Name: accuracy, dtype: object

### Track-level precision per genre for our models

In [53]:
x = []
for type_ in track_new_models_acc:
    x.append(track_new_models_acc[type_]['class_precision'])
    
track_level_models = pd.DataFrame(x)
track_level_models['model'] = pd.Series(['stacked input', 'concatenated architecture'])
track_level_models.set_index('model', inplace=True)
track_level_models

,blues,classical,country,disco,hiphop,jazz,metal,pop,reggae,rock
model,,,,,,,,,,
stacked input,0.666667,1.0,0.318182,0.105263,0.0,0.636364,0.000000,0.500000,0.333333,0.250000
concatenated architecture,0.545455,1.0,0.500000,0.500000,0.4,0.700000,0.833333,0.409091,0.500000,0.285714


### Track-level recall per genre for our models

In [54]:
x = []
for type_ in track_new_models_acc:
    x.append(track_new_models_acc[type_]['class_recall'])
    
track_level_models = pd.DataFrame(x)
track_level_models['model'] = pd.Series(['stacked input', 'concatenated architecture'])
track_level_models.set_index('model', inplace=True)
track_level_models

,blues,classical,country,disco,hiphop,jazz,metal,pop,reggae,rock
model,,,,,,,,,,
stacked input,0.2,0.7,0.7,0.2,0.0,0.7,0.0,0.1,0.8,0.2
concatenated architecture,0.6,0.9,0.4,0.1,0.2,0.7,1.0,0.9,0.7,0.2


In [50]:
with open(git_root('evaluation','evaluation_results','track_new_models_results.json'), 'w') as fp:
    json.dump(track_new_models_acc, fp, indent=4)